In [130]:
url = "http://sejm.gov.pl/Sejm8.nsf/poslowie.xsp?type=A"
MAIN_PATH = "/home/jack/datasets/politycy/"

In [17]:
def flatten(l):
    return [subsub for sub in l for subsub in sub]

import itertools
def head(d):
    return dict(itertools.islice(d.items(), 10))

In [18]:
import requests
from bs4 import BeautifulSoup

In [19]:
page = requests.get(url)

In [20]:
# page.text

In [21]:
bstext = BeautifulSoup(page.text)

/home/jack/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [22]:
politycy = bstext.findAll("div", attrs={"class" : "deputyName"})

In [23]:
linki = {}

for p in politycy:
    linki[p.text] = p.parent.attrs["href"]

In [24]:
# linki

In [25]:
def replace_with_newlines(element):
    text = ''
    for elem in element.recursiveChildGenerator():
        if isinstance(elem, str):
            text += elem.strip()
        elif elem.name == 'br':
            text += '\n'
    return text

In [26]:
from tqdm import tqdm

sejm_url = "http://sejm.gov.pl"

szkola = {}
kluby = {}
def wyjmij_wlasnosc(bst, tag):
    """bst=zupa z beautifulsoup zawierajaca strone posla, tag=tag wlasnosci, ktora chcemy miec"""
    tag_napis = bst.find("p", attrs={"id" : tag})
    tag_parent = tag_napis.parent
    return tag_parent.find("p", attrs={"class" : "right"})

for k in tqdm(linki.keys()):
    sp = requests.get(sejm_url + linki[k]).text
    sp = BeautifulSoup(sp)
    kluby[k] = wyjmij_wlasnosc(sp, "lblLista").text
    szkola[k] = replace_with_newlines(wyjmij_wlasnosc(sp, "lblSzkola"))
#     print(k, szkola[k])

  0%|          | 0/460 [00:00<?, ?it/s]/home/jack/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [27]:
from collections import defaultdict
import re

statsy = defaultdict()
szkola_p = defaultdict()

tytuly = {}

for k, titles in szkola.items():
    tytuly[k] = []
    
    titles = titles.split("\n")
    for title in titles:
        title = title.split(" - ")
        if len(title) > 1:
            title = title[-1]
            title = "".join(title.split("(")[0])
            tytuly[k].append(title.rstrip())
        else:
            if not tytuly[k]:
                tytuly[k].append("nic")

In [28]:
szkola["Kopcińska Joanna"]

'Akademia Medyczna, Wydział Lekarski - lekarz (1997)\nSzkoła Zdrowia Publicznego Instutut Medycyny Pracy im. prof. J. Nofera w Łodzi, Organizacja, zarządzanie i ekonomika w opiece zdrowotnej (2002) - studia podyplomowe\nSzkoła Zdrowia Publicznego Instutut Medycyny Pracy im. prof. J. Nofera w Łodzi, Zdrowie publiczne i środowiskowe (2003) - studia podyplomowe'

In [29]:
import re

p = "Uniwersytet Łódzki, Wydział Ekonomiczno-Socjologiczny, Finanse i bankowość - magister (1996)\n\
Uniwersytet Łódzki, Wydział Ekonomiczno-Socjologiczny, Informatyka i ekonometria - magister (1999)\n\
Uniwersytet Łódzki, Wydział Ekonomiczno-Socjologiczny, Nnauki ekonomiczne - doktor (2000)\n\
Politechnika Łódzka, Pedagogika (2006) - studia podyplomowe"
p2 = "Uniwersytet Łódzki, Wydział Ekonomiczno-Socjologiczny"
print(re.findall("- (\w+\s\w+|\w+)?", p))

['magister', 'magister', 'doktor', 'studia podyplomowe']


In [54]:
from collections import Counter
from pprint import pprint
wszystkie_tyt = Counter(flatten(tytuly.values()))
pprint(wszystkie_tyt)
rodzaje_tyt = {k : 0 for k in wszystkie_tyt.keys()}

Counter({'magister': 319,
         'studia podyplomowe': 268,
         'magister inżynier': 74,
         'doktor': 55,
         'nic': 26,
         'licencjat': 22,
         'technik': 18,
         'lekarz': 17,
         'doktor habilitowany': 17,
         'inżynier': 16,
         'studia doktoranckie': 3,
         'magister inżynier architekt': 3,
         'matura': 2,
         'lekarz weterynarii': 2,
         'dyplomowany ekonomista': 1,
         'lekarz dentysta': 1,
         'magister sztuki': 1})


In [92]:
c = dict(Counter(rodzaje_tyt))
c2 = dict(Counter(["lekarz", "matura", "matura"]))
{**c, **c2}

{'doktor': 0,
 'doktor habilitowany': 0,
 'dyplomowany ekonomista': 0,
 'inżynier': 0,
 'lekarz': 1,
 'lekarz dentysta': 0,
 'lekarz weterynarii': 0,
 'licencjat': 0,
 'magister': 0,
 'magister inżynier': 0,
 'magister inżynier architekt': 0,
 'magister sztuki': 0,
 'matura': 2,
 'nic': 0,
 'studia doktoranckie': 0,
 'studia podyplomowe': 0,
 'technik': 0}

In [32]:
import pandas as pd
poslowie = pd.DataFrame({"wyksztalcenie": list(tytuly.values()), "klub": list(kluby.values())}, index=kluby.keys())

In [35]:
poslowie.head()

,klub,wyksztalcenie
Kwiecień Anna,Prawo i Sprawiedliwość,"[magister, studia podyplomowe]"
Czernow Zofia,Platforma Obywatelska,"[magister, studia podyplomowe, studia podyplom..."
Kubiak Dariusz,Prawo i Sprawiedliwość,[technik]
Jędrysek Mariusz Orion,Prawo i Sprawiedliwość,"[magister, doktor, doktor habilitowany]"
Szulowski Krzysztof,Prawo i Sprawiedliwość,[magister]


In [72]:
kluby_tytuly = poslowie.groupby("klub").aggregate(lambda x: dict(Counter(flatten(x))))

In [73]:
kluby_tytuly_dict_tmp = kluby_tytuly.transpose().to_dict()

In [ ]:
def mergedicts(d1, d2):
    return 

In [152]:
kluby_tytuly_dict = {k : v["wyksztalcenie"] for k, v in kluby_tytuly_dict_tmp.items()}
kluby_tytuly_dict = {k : {**rodzaje_tyt, **v} for k, v  in kluby_tytuly_dict.items()}
kluby_tytuly_df = pd.DataFrame.from_dict(kluby_tytuly_dict).transpose()
kluby_tytuly_df.to_csv(MAIN_PATH + "kluby_tytuly.csv")

In [153]:
kluby_tytuly_df = kluby_tytuly_df.divide(kluby_tytuly_df.sum(axis=1), axis=0)
kluby_tytuly_df.head()

,doktor,doktor habilitowany,dyplomowany ekonomista,inżynier,lekarz,lekarz dentysta,lekarz weterynarii,licencjat,magister,magister inżynier,magister inżynier architekt,magister sztuki,matura,nic,studia doktoranckie,studia podyplomowe,technik
Kukiz'15,0.058824,0.029412,0.0,0.014706,0.000000,0.0,0.029412,0.058824,0.367647,0.029412,0.044118,0.0,0.0,0.088235,0.00000,0.250000,0.029412
Mniejszość Niemiecka,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
Nowoczesna Ryszarda Petru,0.053571,0.035714,0.0,0.000000,0.000000,0.0,0.000000,0.089286,0.428571,0.035714,0.000000,0.0,0.0,0.017857,0.00000,0.321429,0.017857
Platforma Obywatelska,0.068592,0.014440,0.0,0.010830,0.025271,0.0,0.000000,0.021661,0.368231,0.086643,0.000000,0.0,0.0,0.021661,0.00361,0.364621,0.014440
Polskie Stronnictwo Ludowe,0.030303,0.000000,0.0,0.060606,0.000000,0.0,0.000000,0.000000,0.242424,0.151515,0.000000,0.0,0.0,0.000000,0.00000,0.454545,0.060606
